There seems to be a data leakabe problem in the dataset : <br>
https://towardsdatascience.com/distracted-driver-detection-using-deep-learning-e893715e02a4

Going to split on subject instead of randomly. The problem is that the current files do not keep track of the subject

Starting this notebook so that writing the hd5 files are all in one place


In [6]:
classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
#['c'+str(x) for x in range(10)]

class_def = {'c0': 'safe driving',
'c1': 'texting - right',
'c2': 'talking on the phone - right',
'c3': 'texting - left',
'c4': 'talking on the phone - left',
'c5': 'operating the radio',
'c6': 'drinking',
'c7': 'reaching behind',
'c8': 'hair and makeup',
'c9': 'talking to passenger'}

In [7]:
import os
import re
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import random
#Create and load dataset
import h5py

In [8]:
# directories and filepaths
data_dir = Path(os.path.join(os.getcwd(),'data'))
base_dir = Path(os.path.join(data_dir, "state-farm-distracted-driver-detection"))
img_folder = Path(os.path.join(base_dir, 'imgs'))
train_imgs = Path(os.path.join(img_folder, 'train'))
test_imgs = Path(os.path.join(img_folder, 'test'))
# load the image lists
df = pd.read_csv(os.path.join(base_dir, 'driver_imgs_list.csv'))
sample_sub = pd.read_csv(os.path.join(base_dir, 'sample_submission.csv'))

In [9]:
# hdf5 filename 

# CHANGED FILENAMES TO INCLUDE GRAY OR COLOR 
#(THIS WILL BE A DIFF WITH THE LEAKAGE DATASETS THAT DIDNT HAVE DRIVER INFO)
downscale = 2
gray_scale=False
if gray_scale:
    hdf5_train = f"StateFarm_Train_Gray_{downscale}X.h5"
else:
    hdf5_train = f"StateFarm_Train_Color_{downscale}X.h5"

if gray_scale:    
    hdf5_test = f"StateFarm_Test_Gray_{downscale}X.h5"
else:
    hdf5_test = f"StateFarm_Test_Color_{downscale}X.h5"
#"StateFarm_Train_2X.h5" # scaled down by 2
#"StateFarm_Train_5X.h5" # scaled down by 5 

#WRITE (REWRITE) IMAGE HD5 FILES? 
write_dataset = False
write_testset = False 

In [10]:
df.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [11]:
# Look at number of images by subject
#plt.hist(df.subject.value_counts());
df.subject.value_counts()

# There are a few subjects that have a lot more instances than the others 

p021    1237
p022    1233
p024    1226
p026    1196
p016    1078
p066    1034
p049    1011
p051     920
p014     876
p015     875
p035     848
p047     835
p081     823
p012     823
p064     820
p075     814
p061     809
p056     794
p050     790
p052     740
p002     725
p045     724
p039     651
p041     605
p042     591
p072     346
Name: subject, dtype: int64

In [12]:
# conversion from string to array
str2int = lambda istr: int(re.findall('\d+', istr)[0])

df['driver'] =  df.subject.apply(str2int) # SID FOR SUBJECT ID
df['class'] = df.classname.apply(str2int) # 
df.head()

,subject,classname,img,driver,class
0,p002,c0,img_44733.jpg,2,0
1,p002,c0,img_72999.jpg,2,0
2,p002,c0,img_25094.jpg,2,0
3,p002,c0,img_69092.jpg,2,0
4,p002,c0,img_92629.jpg,2,0


In [13]:
# TODO : REMOVE WRITING FILES FROM SVM NOTEBOOK AND MAKE SVM NOTEBOOK ONLY HAVE PCA AND SVM
#
def rescale_image(filepath, resize_scale=2, gray_scale=True):
  '''
  Loads image, converts to grayscale, downsamples by resize_scale and returns a 
  To keep color image, set gray_scale = True 
  To keep the original size of the image, set resize_scale= 1
  '''
  im = Image.open(filepath)
  
  if gray_scale:
    im = im.convert('L')
    
  if resize_scale > 1:
    resize_dims = tuple([int(x/resize_scale) for x in (im.size)])
    im = im.resize(resize_dims)
    
  return np.expand_dims(np.array(im), axis=0)


In [14]:
if write_dataset: 
    # READ AND CONCATENATE TRAINING DATA
    # NOTE: USING NP ARRAYS IS MUCH SLOWER THAN DOING A LIST
    # For training images data set (could probably parallelize this but...)
    classnames = df['classname'].values
    c = df['class'].values
    driver = df['driver'].values
    filenames = df['img'].values
    image_list = [] # make a list, then concatenate
    for i, file in enumerate(filenames):
        if i%1000  == 0:    
            print(f"adding {file} to list, i={i}") 
        path_to_file = Path(os.path.join(train_imgs, classnames[i], file))        
        image_list.append(rescale_image(path_to_file, resize_scale = downscale, gray_scale=gray_scale)) 
    # concatenate images into singe nd array  
    images = np.concatenate(image_list, axis= 0)
    #writehdf5(data_dir, hdf5_train, images, labels)

In [15]:
if write_dataset: 
    print('Writing HDF5 file')
    # Create a new HDF5 file
    file = h5py.File(
        data_dir / hdf5_train, "w"
    )

    # Create a dataset in the file
    image_set = file.create_dataset(
        "images",
        np.shape(images),
        h5py.h5t.STD_U8BE,
        data=images,
    )
    classnames_set = file.create_dataset(
        "c",
        np.shape(c),
        h5py.h5t.STD_U8BE,
        data=c,
    )    
    
    driver_set = file.create_dataset(
        "driver",
        np.shape(driver),
        h5py.h5t.STD_U8BE,
        data=driver,
    ) 
    
    file_set = file.create_dataset("fname", data=np.array(filenames, dtype='S'))
    
    file.close()      

In [16]:
images.shape

NameError: name 'images' is not defined

In [56]:
# We want to split on the person id 
# However, not sure how the classes are represented by driver
drivers = np.unique(df['driver'].values)
ndrivers = len(drivers)

for i in range(ndrivers):
    print(df[df['driver']==drivers[i]].groupby(['driver', 'class']).count())
    


              subject  classname  img
driver class                         
2      0           76         76   76
       1           74         74   74
       2           86         86   86
       3           79         79   79
       4           84         84   84
       5           76         76   76
       6           83         83   83
       7           72         72   72
       8           44         44   44
       9           51         51   51
              subject  classname  img
driver class                         
12     0           84         84   84
       1           95         95   95
       2           91         91   91
       3           89         89   89
       4           97         97   97
       5           96         96   96
       6           75         75   75
       7           72         72   72
       8           62         62   62
       9           62         62   62
              subject  classname  img
driver class                         
14     0    

In [71]:
# HOW TO SPLIT DATASETS BY DRIVER: 

# All the drivers have data in all the classes. So that's good. 
# Still, because 50 and 51 seem to be really safe drivers, 
# going to split on a shuffle of the drivers

# wanting to do roughly 80/20 split but keeping the drivers separate
# get npts per driver
nPerDriver = df.groupby('driver').count().values
nPerDriver = nPerDriver[:,0]
driver_idx  = np.random.permutation(ndrivers)
# shuffle the drivers
drivers = drivers[driver_idx]
nPerDriver = nPerDriver[driver_idx]

In [85]:
train_driver = np.cumsum(nPerDriver) < (22424*.8)
test_driver = np.logical_not(train_driver)

SyntaxError: invalid syntax (Temp/ipykernel_3572/84762919.py, line 1)